In [1]:
from splinter import Browser
import time 
import numpy as np
import requests
import pandas as pd
from collections import Counter
import os
import logging

In [ ]:
SAVING_PATH = '/Users/amr/Downloads/MIT_LA/'
COURSE_URL = ''

### Helper functions 

In [ ]:
def local_name_check(saving_name):
    '''
    A function to avoid overwriting
    Input: desired saving name
    Output: the same name with an additional "_1" suffiex if the same file name already existed
    '''
    
    if os.path.isfile(saving_name):
        file_extension = saving_name[::-1].split('.', 1)[0][::-1]
        print(file_extension)
        saving_name = saving_name.strip('.'+file_extension) + "_1" + "."+file_extension
        print(saving_name)
    return saving_name


# source:
# https://www.codementor.io/aviaryan/downloading-files-from-urls-in-python-77q3bs0un

def is_downloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

def download_file(url, saving_name):
    '''
    Downloads a file to the "saving_name" directory
    Returns the final saved name
    '''
    
    saving_name = local_name_check(saving_name)
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True, allow_redirects=True)
    
    with open(saving_name, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return saving_name

### Scrape videos' URLs

In [82]:
browser = Browser("chrome")
browser.visit(COURSE_URL)

In [83]:

vids_pages = []

# scrape all links to videos' pages until "Next" button no more exist.
while True:
    
    vids = browser.find_by_css("#middle > div > p > span > a")
    vids_pages.extend([(x.text, x['href']) for x in vids])
    
    # keep adding videos if there is 
    if browser.is_element_present_by_css('''#middle > div > div.diytube-pager > span > a.diytube-pager-next'''):
        browser.find_by_css('''#middle > div > div.diytube-pager > span > a.diytube-pager-next''').click()
    else:
        break;
    

### Download videos

In [1]:
downloaded_vids = []
failed_downloads = []

# browser = Browser("chrome")


for i, (video_name, video_page_link) in enumerate(vids_pages[::-1]):
    
    try:
        browser.visit(video_page_link)
        
        # There is a 255 max character limit on mac file system, 
        # also "/" inside a file name could interpreted as a folder name seperator
        saving_name = SAVING_PATH + str(i) + "_" + re.sub('''[\(\)\/\\:]''', "", video_name[:200])+ ".mp4"
        text_box = browser.find_by_css('''#middle > div > div.diytube-embed-html''').text
        downloadable_vid = text_box[text_box.find('src') + 5:].split('\"')[0]
        print('Downloading: ', downloadable_vid)
        if is_downloadable(downloadable_vid):
            saving_name = download_file(downloadable_vid, saving_name)
            downloaded_vids.append((saving_name, downloadable_vid))
            
    except Exception as e:
        logging.error('Failed to download a file, '+ str(e))
        failed_downloads.append((video_name, video_page_link))